# Predictive Scaling & Capacity Planning

## Overview
This notebook implements predictive scaling and capacity planning. It forecasts resource demand, triggers proactive scaling, and optimizes resource allocation across the platform.

**Model Serving**: This notebook trains and saves a sklearn model to the shared PVC (`/mnt/models/`) for the `predictive-analytics` InferenceService. The model is saved during both validation and manual execution.

## Prerequisites
- Completed: `multi-cluster-healing-coordination.ipynb`
- Historical resource usage data
- Kubernetes metrics available
- HPA (Horizontal Pod Autoscaler) configured

## Learning Objectives
- Forecast resource demand
- Implement predictive scaling
- Plan capacity requirements
- Optimize resource allocation
- Prevent resource exhaustion

## Key Concepts
- **Demand Forecasting**: Predict future resource needs
- **Predictive Scaling**: Scale before demand spike
- **Capacity Planning**: Allocate resources efficiently
- **Resource Optimization**: Minimize waste
- **Cost Efficiency**: Balance performance and cost

## Setup Section

In [ ]:
import sys
import os
import json
import logging
import joblib
from pathlib import Path
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from typing import Dict, List, Any

# Setup path for utils module - works from any directory
def find_utils_path():
    """Find utils path regardless of current working directory"""
    possible_paths = [
        Path(__file__).parent.parent / 'utils' if '__file__' in dir() else None,
        Path.cwd() / 'notebooks' / 'utils',
        Path.cwd().parent / 'utils',
        Path('/workspace/repo/notebooks/utils'),
        Path('/opt/app-root/src/notebooks/utils'),
        Path('/opt/app-root/src/openshift-aiops-platform/notebooks/utils'),
    ]
    for p in possible_paths:
        if p and p.exists() and (p / 'common_functions.py').exists():
            return str(p)
    current = Path.cwd()
    for _ in range(5):
        utils_path = current / 'notebooks' / 'utils'
        if utils_path.exists():
            return str(utils_path)
        current = current.parent
    return None

utils_path = find_utils_path()
if utils_path:
    sys.path.insert(0, utils_path)
    print(f"✅ Utils path found: {utils_path}")
else:
    print("⚠️ Utils path not found - will use fallback implementations")

# Try to import common functions, with fallback
try:
    from common_functions import setup_environment
    print("✅ Common functions imported")
except ImportError as e:
    print(f"⚠️ Common functions not available: {e}")
    def setup_environment():
        os.makedirs('/opt/app-root/src/data/processed', exist_ok=True)
        os.makedirs('/opt/app-root/src/models', exist_ok=True)
        return {'data_dir': '/opt/app-root/src/data', 'models_dir': '/opt/app-root/src/models'}

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Setup environment
env_info = setup_environment()
logger.info(f"Environment ready: {env_info}")

# Define paths
DATA_DIR = Path('/opt/app-root/src/data')
PROCESSED_DIR = DATA_DIR / 'processed'
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

# Model storage paths
# Primary: PVC mount for KServe InferenceService (predictive-analytics)
# Fallback: Local models directory
PVC_MODELS_DIR = Path('/mnt/models')
LOCAL_MODELS_DIR = Path('/opt/app-root/src/models')
LOCAL_MODELS_DIR.mkdir(parents=True, exist_ok=True)

# Configuration
NAMESPACE = 'self-healing-platform'
FORECAST_HORIZON = 24  # 24 hours ahead
SCALING_THRESHOLD = 0.80  # Scale at 80% capacity

logger.info(f"Predictive scaling initialized")
logger.info(f"PVC models directory: {PVC_MODELS_DIR} (exists: {PVC_MODELS_DIR.exists()})")
logger.info(f"Local models directory: {LOCAL_MODELS_DIR}")

## Implementation Section

### 1. Generate Training Data and Train Predictive Model

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

class PredictiveScalingEnsemble(BaseEstimator, TransformerMixin):
    """
    Wrapper class that combines CPU and Memory prediction models for KServe compatibility.
    KServe requires a single .pkl file with a predict() method, not a dictionary.
    """
    def __init__(self, cpu_model=None, memory_model=None, feature_cols=None, 
                 trained_at=None, training_samples=None, model_version='1.0.0'):
        self.cpu_model = cpu_model
        self.memory_model = memory_model
        self.feature_cols = feature_cols or []
        self.trained_at = trained_at
        self.training_samples = training_samples
        self.model_version = model_version

    def predict(self, X):
        """
        Make predictions using both CPU and Memory models.
        
        Args:
            X: Input features (hour_of_day, day_of_week, cpu_rolling_mean, memory_rolling_mean)
        
        Returns:
            Dictionary with CPU and Memory predictions
        """
        results = {}
        
        if self.cpu_model is not None:
            try:
                cpu_pred = self.cpu_model.predict(X)
                results['cpu_forecast'] = cpu_pred.tolist() if hasattr(cpu_pred, 'tolist') else cpu_pred
            except Exception as e:
                results['cpu_error'] = str(e)
                logger.warning(f"CPU prediction failed: {e}")
        
        if self.memory_model is not None:
            try:
                memory_pred = self.memory_model.predict(X)
                results['memory_forecast'] = memory_pred.tolist() if hasattr(memory_pred, 'tolist') else memory_pred
            except Exception as e:
                results['memory_error'] = str(e)
                logger.warning(f"Memory prediction failed: {e}")
        
        # Return combined results
        return results
    
    def get_params(self, deep=True):
        return {
            'cpu_model': self.cpu_model,
            'memory_model': self.memory_model,
            'feature_cols': self.feature_cols,
            'trained_at': self.trained_at,
            'training_samples': self.training_samples,
            'model_version': self.model_version
        }
    
    def set_params(self, **params):
        for key, value in params.items():
            setattr(self, key, value)
        return self

print("✅ PredictiveScalingEnsemble class defined for KServe compatibility")

In [ ]:
# Generate realistic historical resource usage data
np.random.seed(42)  # For reproducibility

# Create 7 days of hourly data with realistic patterns
hours = 168  # 7 days
timestamps = [datetime.now() - timedelta(hours=i) for i in range(hours, 0, -1)]

# Simulate daily patterns (higher during business hours)
hour_of_day = np.array([t.hour for t in timestamps])
day_of_week = np.array([t.weekday() for t in timestamps])

# Base load + daily pattern + weekly pattern + noise
base_cpu = 0.4
daily_pattern = 0.2 * np.sin(2 * np.pi * (hour_of_day - 6) / 24)  # Peak at 2pm
weekly_pattern = 0.1 * (1 - day_of_week / 7)  # Higher on weekdays
noise = np.random.normal(0, 0.05, hours)

cpu_usage = np.clip(base_cpu + daily_pattern + weekly_pattern + noise, 0.1, 0.95)
memory_usage = np.clip(cpu_usage * 1.1 + np.random.normal(0, 0.03, hours), 0.2, 0.95)

historical_data = pd.DataFrame({
    'timestamp': timestamps,
    'hour_of_day': hour_of_day,
    'day_of_week': day_of_week,
    'cpu_usage': cpu_usage,
    'memory_usage': memory_usage
})

print(f"Generated {len(historical_data)} hours of historical data")
print(f"CPU usage range: {cpu_usage.min():.2%} - {cpu_usage.max():.2%}")
print(f"Memory usage range: {memory_usage.min():.2%} - {memory_usage.max():.2%}")

# Train a predictive model for resource forecasting
# Features: hour_of_day, day_of_week, rolling averages
historical_data['cpu_rolling_mean'] = historical_data['cpu_usage'].rolling(window=24, min_periods=1).mean()
historical_data['memory_rolling_mean'] = historical_data['memory_usage'].rolling(window=24, min_periods=1).mean()

# Prepare features and target
feature_cols = ['hour_of_day', 'day_of_week', 'cpu_rolling_mean', 'memory_rolling_mean']
X = historical_data[feature_cols].values
y_cpu = historical_data['cpu_usage'].values
y_memory = historical_data['memory_usage'].values

# Create sklearn pipeline for CPU prediction
cpu_model = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', RandomForestRegressor(n_estimators=50, max_depth=10, random_state=42))
])
cpu_model.fit(X, y_cpu)

# Create sklearn pipeline for Memory prediction
memory_model = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', RandomForestRegressor(n_estimators=50, max_depth=10, random_state=42))
])
memory_model.fit(X, y_memory)

# Combine into a single sklearn-compatible object (KServe requirement)
predictive_model = PredictiveScalingEnsemble(
    cpu_model=cpu_model,
    memory_model=memory_model,
    feature_cols=feature_cols,
    trained_at=datetime.now().isoformat(),
    training_samples=len(historical_data),
    model_version='1.0.0'
)

logger.info(f"✅ Trained predictive scaling model with {len(historical_data)} samples")

### 2. Save Model to PVC for KServe InferenceService

In [ ]:
import subprocess
import pickle
import io

# ✨ Use KServe-compatible subdirectory structure
# KServe sklearn server expects: /mnt/models/predictive-analytics/model.pkl
MODEL_NAME = 'predictive-analytics'

def save_model_for_kserve(model, model_name='predictive-analytics'):
    """
    Save sklearn model for KServe InferenceService with correct subdirectory structure.
    
    KServe Requirements:
    - Model must be in subdirectory: /mnt/models/{model_name}/
    - Exactly ONE .pkl file: model.pkl
    - storageUri: "pvc://model-storage-pvc/predictive-analytics"
    
    Args:
        model: Trained sklearn model/pipeline (PredictiveScalingEnsemble)
        model_name: Name for the model subdirectory (default: 'predictive-analytics')
    
    Returns:
        Dict with save status and paths
    """
    results = {'saved_to': [], 'errors': [], 'kserve_restarted': False}
    
    # Option 1: Save to PVC with KServe subdirectory structure (if mounted)
    if PVC_MODELS_DIR.exists():
        try:
            # Create model subdirectory
            model_dir = PVC_MODELS_DIR / model_name
            model_dir.mkdir(parents=True, exist_ok=True)
            
            # Remove old flat file if it exists (migration)
            old_flat_file = PVC_MODELS_DIR / 'model.pkl'
            if old_flat_file.exists():
                old_flat_file.unlink()
                logger.info(f"🗑️  Removed old flat file: {old_flat_file}")
            
            # Save to KServe-compatible path
            pvc_model_path = model_dir / 'model.pkl'
            joblib.dump(model, pvc_model_path)
            results['saved_to'].append(str(pvc_model_path))
            logger.info(f"✅ Model saved to PVC: {pvc_model_path}")
            
            # Verify only ONE .pkl file exists (KServe requirement)
            pkl_files = list(model_dir.glob('*.pkl'))
            if len(pkl_files) != 1:
                raise RuntimeError(
                    f"❌ ERROR: Expected 1 .pkl file, found {len(pkl_files)}: {pkl_files}\n"
                    f"KServe requires EXACTLY ONE .pkl file per model directory."
                )
            
            print(f"✅ KServe validation passed:")
            print(f"   Path: {pvc_model_path}")
            print(f"   Files in directory: {len(pkl_files)} (correct - must be 1)")
            
            # Save metadata - access ensemble attributes
            metadata = {
                'model_name': model_name,
                'saved_at': datetime.now().isoformat(),
                'model_type': 'sklearn_predictive_model',
                'features': model.feature_cols if hasattr(model, 'feature_cols') else [],
                'version': model.model_version if hasattr(model, 'model_version') else '1.0.0',
                'kserve_compatible': True
            }
            metadata_path = model_dir / 'metadata.json'
            with open(metadata_path, 'w') as f:
                json.dump(metadata, f, indent=2)
            
            # Trigger rollout restart so KServe picks up the new model
            try:
                restart_result = subprocess.run(
                    ["oc", "rollout", "restart", "deployment/predictive-analytics-predictor-00001-deployment", 
                     "-n", NAMESPACE],
                    capture_output=True, text=True, timeout=30
                )
                if restart_result.returncode == 0:
                    results['kserve_restarted'] = True
                    logger.info(f"✅ Triggered rollout restart of predictive-analytics-predictor")
                else:
                    logger.warning(f"⚠️ Could not restart predictor: {restart_result.stderr.strip()}")
            except Exception as restart_err:
                logger.warning(f"⚠️ Rollout restart skipped: {restart_err}")
                
        except Exception as e:
            results['errors'].append(f"PVC save failed: {e}")
            logger.warning(f"⚠️ PVC save failed: {e}")
    else:
        logger.info(f"ℹ️ PVC not mounted at {PVC_MODELS_DIR}")
        results['errors'].append(f"PVC not mounted at {PVC_MODELS_DIR}")
    
    # Option 2: Always save to local models directory (fallback)
    try:
        # Use subdirectory structure locally too
        local_model_dir = LOCAL_MODELS_DIR / model_name
        local_model_dir.mkdir(parents=True, exist_ok=True)
        local_model_path = local_model_dir / 'model.pkl'
        joblib.dump(model, local_model_path)
        results['saved_to'].append(str(local_model_path))
        logger.info(f"✅ Model saved locally: {local_model_path}")
    except Exception as e:
        results['errors'].append(f"Local save failed: {e}")
        logger.error(f"❌ Local save failed: {e}")
    
    return results

# Save the predictive model
print(f"💾 Saving predictive model for KServe InferenceService...")
print(f"   Model name: {MODEL_NAME}")
print(f"   Expected KServe path: /mnt/models/{MODEL_NAME}/model.pkl")

save_results = save_model_for_kserve(predictive_model, MODEL_NAME)

print(f"\n📊 Model Save Results:")
print(f"  Saved to: {save_results['saved_to']}")
print(f"  KServe predictor restarted: {'✅ Yes' if save_results['kserve_restarted'] else '⚠️ No'}")
if save_results['errors']:
    print(f"  Errors: {save_results['errors']}")
else:
    print(f"  ✅ Ready for KServe deployment!")
    print(f"  Deploy with: storageUri: pvc://model-storage-pvc/{MODEL_NAME}")

### 3. Forecast Resource Demand

In [ ]:
def forecast_demand(model, current_data: pd.DataFrame, horizon: int = 24) -> Dict[str, Any]:
    """
    Forecast resource demand using trained model.
    
    Args:
        model: Trained PredictiveScalingEnsemble object
        current_data: Current resource usage data
        horizon: Forecast horizon in hours
    
    Returns:
        Demand forecast
    """
    try:
        # Access models from ensemble object
        cpu_model = model.cpu_model
        memory_model = model.memory_model
        
        # Generate future timestamps
        future_timestamps = [datetime.now() + timedelta(hours=i) for i in range(1, horizon + 1)]
        
        # Create features for prediction
        future_features = []
        last_cpu_mean = current_data['cpu_usage'].tail(24).mean()
        last_memory_mean = current_data['memory_usage'].tail(24).mean()
        
        for ts in future_timestamps:
            future_features.append([
                ts.hour,
                ts.weekday(),
                last_cpu_mean,
                last_memory_mean
            ])
        
        X_future = np.array(future_features)
        
        # Predict
        cpu_forecast = cpu_model.predict(X_future)
        memory_forecast = memory_model.predict(X_future)
        
        forecast = {
            'timestamp': datetime.now().isoformat(),
            'horizon_hours': horizon,
            'cpu_forecast': cpu_forecast.tolist(),
            'memory_forecast': memory_forecast.tolist(),
            'peak_cpu': float(np.max(cpu_forecast)),
            'peak_memory': float(np.max(memory_forecast)),
            'avg_cpu': float(np.mean(cpu_forecast)),
            'avg_memory': float(np.mean(memory_forecast))
        }
        
        logger.info(f"Demand forecast: Peak CPU {forecast['peak_cpu']:.1%}, Peak Memory {forecast['peak_memory']:.1%}")
        return forecast
    except Exception as e:
        logger.error(f"Demand forecasting error: {e}")
        return {'error': str(e)}

# Generate forecast
forecast = forecast_demand(predictive_model, historical_data, FORECAST_HORIZON)
print(json.dumps({k: v for k, v in forecast.items() if k not in ['cpu_forecast', 'memory_forecast']}, indent=2, default=str))

### 4. Trigger Predictive Scaling

In [ ]:
def trigger_predictive_scaling(forecast: Dict[str, Any], current_replicas: int) -> Dict[str, Any]:
    """
    Trigger predictive scaling based on forecast.
    
    Args:
        forecast: Demand forecast
        current_replicas: Current number of replicas
    
    Returns:
        Scaling decision
    """
    try:
        peak_cpu = forecast.get('peak_cpu', 0)
        
        # Calculate required replicas
        if peak_cpu > SCALING_THRESHOLD:
            # Scale up: add 20% more replicas
            required_replicas = int(current_replicas * (peak_cpu / SCALING_THRESHOLD))
            scaling_action = 'scale_up'
        elif peak_cpu < 0.5:
            # Scale down: reduce by 20%
            required_replicas = max(1, int(current_replicas * 0.8))
            scaling_action = 'scale_down'
        else:
            required_replicas = current_replicas
            scaling_action = 'no_change'
        
        scaling_decision = {
            'timestamp': datetime.now().isoformat(),
            'current_replicas': current_replicas,
            'required_replicas': required_replicas,
            'scaling_action': scaling_action,
            'peak_cpu_forecast': peak_cpu,
            'scaling_triggered': scaling_action != 'no_change',
            'estimated_cost_savings': (current_replicas - required_replicas) * 100 if scaling_action == 'scale_down' else 0
        }
        
        logger.info(f"Scaling decision: {scaling_action} ({current_replicas} -> {required_replicas})")
        return scaling_decision
    except Exception as e:
        logger.error(f"Predictive scaling error: {e}")
        return {'error': str(e)}

# Test scaling decision
scaling_decision = trigger_predictive_scaling(forecast, current_replicas=5)
print(json.dumps(scaling_decision, indent=2, default=str))

### 5. Capacity Planning

In [ ]:
def plan_capacity(forecast: Dict[str, Any], current_capacity: Dict[str, float]) -> Dict[str, Any]:
    """
    Plan capacity requirements based on forecast.
    
    Args:
        forecast: Demand forecast
        current_capacity: Current capacity allocation
    
    Returns:
        Capacity plan
    """
    try:
        peak_cpu = forecast.get('peak_cpu', 0)
        peak_memory = forecast.get('peak_memory', 0)
        
        # Calculate required capacity with 20% headroom
        required_cpu = peak_cpu * 1.2
        required_memory = peak_memory * 1.2
        
        capacity_plan = {
            'timestamp': datetime.now().isoformat(),
            'current_cpu_capacity': current_capacity.get('cpu', 0),
            'required_cpu_capacity': required_cpu,
            'cpu_headroom': required_cpu - current_capacity.get('cpu', 0),
            'current_memory_capacity': current_capacity.get('memory', 0),
            'required_memory_capacity': required_memory,
            'memory_headroom': required_memory - current_capacity.get('memory', 0),
            'capacity_sufficient': (required_cpu <= current_capacity.get('cpu', 0) and 
                                   required_memory <= current_capacity.get('memory', 0)),
            'recommendations': []
        }
        
        # Generate recommendations
        if capacity_plan['cpu_headroom'] > 0:
            capacity_plan['recommendations'].append(
                f"Add {capacity_plan['cpu_headroom']:.1f} CPU cores"
            )
        if capacity_plan['memory_headroom'] > 0:
            capacity_plan['recommendations'].append(
                f"Add {capacity_plan['memory_headroom']:.1f}GB memory"
            )
        
        logger.info(f"Capacity plan: {len(capacity_plan['recommendations'])} recommendations")
        return capacity_plan
    except Exception as e:
        logger.error(f"Capacity planning error: {e}")
        return {'error': str(e)}

# Test capacity planning
current_capacity = {'cpu': 16, 'memory': 64}
capacity_plan = plan_capacity(forecast, current_capacity)
print(json.dumps(capacity_plan, indent=2, default=str))

### 6. Track Scaling History

In [ ]:
# Create scaling tracking dataframe
scaling_tracking = pd.DataFrame([
    {
        'timestamp': datetime.now() - timedelta(hours=i),
        'current_replicas': np.random.randint(3, 10),
        'target_replicas': np.random.randint(3, 10),
        'scaling_action': np.random.choice(['scale_up', 'scale_down', 'no_change']),
        'forecast_accuracy': np.random.uniform(0.75, 0.95),
        'cost_savings': np.random.uniform(0, 500),
        'performance_impact': np.random.choice(['positive', 'neutral', 'negative'])
    }
    for i in range(168)  # 7 days of data
])

# Save tracking data
tracking_file = PROCESSED_DIR / 'predictive_scaling_tracking.parquet'
scaling_tracking.to_parquet(tracking_file)

logger.info(f"Saved predictive scaling tracking data")
print(f"Tracking data saved to: {tracking_file}")
print(f"Records: {len(scaling_tracking)}")

## Validation Section

In [ ]:
# Verify outputs
assert tracking_file.exists(), "Scaling tracking file not created"
assert 'peak_cpu' in forecast, "No CPU forecast"
assert 'scaling_action' in scaling_decision, "No scaling decision"
assert len(save_results['saved_to']) > 0, "Model not saved to any location"

# Check if model was saved to PVC (critical for KServe)
pvc_model_exists = (PVC_MODELS_DIR / 'model.pkl').exists() if PVC_MODELS_DIR.exists() else False
local_model_exists = (LOCAL_MODELS_DIR / 'model.pkl').exists()
kserve_restarted = save_results.get('kserve_restarted', False)

avg_forecast_accuracy = scaling_tracking['forecast_accuracy'].mean()
total_cost_savings = scaling_tracking['cost_savings'].sum()
scale_up_count = (scaling_tracking['scaling_action'] == 'scale_up').sum()

logger.info(f"✅ All validations passed")
print(f"\n{'='*60}")
print(f"Predictive Scaling & Capacity Planning Summary")
print(f"{'='*60}")
print(f"\n📊 Training Data:")
print(f"   Historical Records: {len(historical_data)}")
print(f"   Features: {predictive_model.feature_cols}")
print(f"\n🤖 Model Status:")
print(f"   PVC Model (/mnt/models/model.pkl): {'✅ Saved' if pvc_model_exists else '❌ Not saved (PVC not mounted)'}")
print(f"   Local Model: {'✅ Saved' if local_model_exists else '❌ Not saved'}")
print(f"   KServe Predictor Restarted: {'✅ Yes' if kserve_restarted else '⚠️ No (deployment may not exist yet)'}")
print(f"   KServe Ready: {'✅ Yes' if pvc_model_exists else '⚠️ No - run in workbench with PVC mounted'}")
print(f"\n📈 Forecast Results:")
print(f"   Forecast Horizon: {FORECAST_HORIZON} hours")
print(f"   Peak CPU: {forecast['peak_cpu']:.1%}")
print(f"   Peak Memory: {forecast['peak_memory']:.1%}")
print(f"\n⚡ Scaling Decision:")
print(f"   Action: {scaling_decision['scaling_action']}")
print(f"   Replicas: {scaling_decision['current_replicas']} -> {scaling_decision['required_replicas']}")
print(f"\n📉 Historical Stats:")
print(f"   Tracking Records: {len(scaling_tracking)}")
print(f"   Average Forecast Accuracy: {avg_forecast_accuracy:.1%}")
print(f"   Total Cost Savings: ${total_cost_savings:.0f}")
print(f"   Scale-Up Events: {scale_up_count}")

## Integration Section

This notebook integrates with:
- **Input**: Historical resource usage and forecasts
- **Output**: 
  - Trained sklearn model saved to PVC (`/mnt/models/model.pkl`)
  - Model served by `predictive-analytics` InferenceService
  - Scaling decisions and capacity plans
- **Monitoring**: Forecast accuracy and cost savings
- **Next**: Security incident response automation

### Model Serving

The trained model is automatically available to the `predictive-analytics` InferenceService:

```yaml
apiVersion: serving.kserve.io/v1beta1
kind: InferenceService
metadata:
  name: predictive-analytics
spec:
  predictor:
    model:
      modelFormat:
        name: sklearn
      storageUri: "pvc://model-storage-pvc"
```

## Next Steps

1. ✅ Model deployed to PVC for KServe
2. Verify `predictive-analytics` InferenceService is ready
3. Proceed to `security-incident-response-automation.ipynb`
4. Implement security incident handling
5. Automate response procedures

## References

- ADR-003: Self-Healing Platform Architecture
- ADR-012: Notebook Architecture for End-to-End Workflows
- [Kubernetes HPA](https://kubernetes.io/docs/tasks/run-application/horizontal-pod-autoscale/)
- [Capacity Planning](https://en.wikipedia.org/wiki/Capacity_planning)